In [1]:
import multimodal_cci as mmcci
import scanpy as sc
import numpy as np

#### Load stLearn data objects

In [ ]:
brain_a_vis = [sc.read_h5ad("/QRISdata/Q5058/sMACS/Visium/Processed/Brain/CCI/VLP40_1C_CCI.h5ad"),
                sc.read_h5ad("/QRISdata/Q5058/sMACS/Visium/Processed/Brain/CCI/VLP40_1D_CCI.h5ad"),
                sc.read_h5ad("/QRISdata/Q5058/sMACS/Visium/Processed/Brain/CCI/VLP41_1B_CCI.h5ad"),
                sc.read_h5ad("/QRISdata/Q5058/sMACS/Visium/Processed/Brain/CCI/VLP41_1D_CCI.h5ad")]
brain_y_vis = [sc.read_h5ad("/QRISdata/Q5058/sMACS/Visium/Processed/Brain/CCI/VLP40_1A_CCI.h5ad"),
               sc.read_h5ad("/QRISdata/Q5058/sMACS/Visium/Processed/Brain/CCI/VLP40_1B_CCI.h5ad"),
               sc.read_h5ad("/QRISdata/Q5058/sMACS/Visium/Processed/Brain/CCI/VLP41_1A_CCI.h5ad"),
               sc.read_h5ad("/QRISdata/Q5058/sMACS/Visium/Processed/Brain/CCI/VLP41_1C_CCI.h5ad")]

brain_a_sto = [sc.read_h5ad("/QRISdata/Q5058/sMACS/STomics/Processed/Brain/CCI_new/brain_A1_CCI_bin80.h5ad"),
               sc.read_h5ad("/QRISdata/Q5058/sMACS/STomics/Processed/Brain/CCI_new/brain_A4_CCI_bin80.h5ad")]
brain_y_sto = [sc.read_h5ad("/QRISdata/Q5058/sMACS/STomics/Processed/Brain/CCI_new/brain_Y1_CCI_bin80.h5ad"),
               sc.read_h5ad("/QRISdata/Q5058/sMACS/STomics/Processed/Brain/CCI_new/brain_Y4_CCI_bin80.h5ad")]

#### Normalise each group of samples

In [ ]:
brain_a_vis_normalised = mmcci.it.normalise_samples_to_target(brain_a_vis, target=brain_y_vis[0])
brain_y_vis_normalised = mmcci.it.normalise_samples_to_target(brain_y_vis, target=brain_y_vis[0])
brain_a_sto_normalised = mmcci.it.normalise_samples_to_target(brain_a_sto, target=brain_y_sto[0])
brain_y_sto_normalised = mmcci.it.normalise_samples_to_target(brain_y_sto, target=brain_y_sto[0])

#### Subset to only LR pairs present in majority of samples

In [ ]:
aged_lr_pairs = mmcci.it.get_majority_lr_pairs(brain_a_vis_normalised + brain_a_sto_normalised)
young_lr_pairs = mmcci.it.get_majority_lr_pairs(brain_y_vis_normalised + brain_y_sto_normalised)

all_lr_pairs = aged_lr_pairs + young_lr_pairs

#### Calculate average value of each LR pair across each group

In [ ]:
brain_a_vis_avg = mmcci.it.get_avg_lr_pairs(brain_a_vis, all_lr_pairs)
brain_y_vis_avg = mmcci.it.get_avg_lr_pairs(brain_y_vis, all_lr_pairs)
brain_a_sto_avg = mmcci.it.get_avg_lr_pairs(brain_a_sto, all_lr_pairs)
brain_y_sto_avg = mmcci.it.get_avg_lr_pairs(brain_y_sto, all_lr_pairs)

#### Normalise samples between technologies

In [ ]:
normalised = mmcci.it.normalise_samples_between_tech([[brain_a_vis_avg, brain_y_vis_avg],
                                             [brain_a_sto_avg, brain_y_sto_avg]])

#### Integrate between technologies for both groups

In [ ]:
normalised = np.array(normalised).T.tolist() # Transpose to get lists of groups instead of techs
aged_integrated = mmcci.it.integrate_between_tech(normalised[0])
young_integrated = mmcci.it.integrate_between_tech(normalised[1])

#### Calculate overall interactions network per group

In [ ]:
aged_overall = mmcci.it.calculate_overall_interactions(aged_integrated)
young_overall = mmcci.it.calculate_overall_interactions(young_integrated)

#### Rank LR pairs based on dissimilarity between groups

In [ ]:
dissimilarity_scores = mmcci.it.calculate_dissim(aged_integrated, young_integrated)
mmcci.plt.dissim_hist(dissimilarity_scores)
    
## Sort LR Pairs from high to low dissimilarities
print("Overall Network Aged vs Young Dissimilarity Score is: " + str(mmcci.sc.dissimilarity_score(aged_overall, young_overall)))

sorted_pairs = sorted(dissimilarity_scores, key=dissimilarity_scores.get, reverse=True)
print("\nTop 10 Dissimilar LR Pairs:", sorted_pairs[:10])

#### Plot highly dissimilar pairs between groups

In [ ]:
mmcci.plt.network_plot(aged_integrated['A2m_Lrp1'])
mmcci.plt.network_plot(young_integrated['A2m_Lrp1'])

#### Calculate p values for significance changes between groups

In [ ]:
p_values = mmcci.sc.perm_test(aged_overall, young_overall)
p_values

#### Plot aged and young overall networks

In [ ]:
mmcci.plt.network_plot(aged_overall, p_values)
mmcci.plt.network_plot(young_overall, p_values)

#### Get LR pairs from Vascular to Astrocytes

In [ ]:
vasc_astr_lrs = mmcci.it.get_lrs_per_celltype([aged_integrated, young_integrated], "Vascular", "Astrocytes")
print("LR pairs from vascular to astrocytes in aged")
print(vasc_astr_lrs['sample1'])
print("\nLR pairs from vascular to astrocytes in young")
print(vasc_astr_lrs['sample2'])

#### Cluster aged and young lr pairs

In [ ]:
aged_clusters = mmcci.it.lr_clustering(aged_integrated, n_clusters=0)
aged_clusters

In [ ]:
young_clusters = mmcci.it.lr_clustering(young_integrated, n_clusters=0)
young_clusters